In [10]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import re, glob
import os, sys
from scipy import stats
import csv


In [40]:
# read the covid data file
covid_df = pd.read_csv("Resources/owid_covid_data.csv")                
 # Filter for country of your choice

covid_df.head(20)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
5,AFG,Asia,Afghanistan,2020-02-29,1.0,0.0,0.143,NaN,NaN,0.0,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
6,AFG,Asia,Afghanistan,2020-03-01,1.0,0.0,0.143,NaN,NaN,0.0,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
7,AFG,Asia,Afghanistan,2020-03-02,1.0,0.0,0.000,NaN,NaN,0.0,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
8,AFG,Asia,Afghanistan,2020-03-03,2.0,1.0,0.143,NaN,NaN,0.0,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
9,AFG,Asia,Afghanistan,2020-03-04,4.0,2.0,0.429,NaN,NaN,0.0,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [41]:
new_df = covid_df.loc[:,["location","date","stringency_index","life_expectancy","human_development_index"]]


In [42]:
new_df = new_df.dropna()
new_df.head()

,location,date,stringency_index,life_expectancy,human_development_index
0,Afghanistan,2020-02-24,8.33,64.83,0.511
1,Afghanistan,2020-02-25,8.33,64.83,0.511
2,Afghanistan,2020-02-26,8.33,64.83,0.511
3,Afghanistan,2020-02-27,8.33,64.83,0.511
4,Afghanistan,2020-02-28,8.33,64.83,0.511


In [53]:
grouped_df= new_df.groupby("stringency_index").agg('max')

In [63]:
top_higher_stringency = grouped_df.sort_values(by=['stringency_index'], ascending=False)
top_higher_stringency.head(10)

,location,date,life_expectancy,human_development_index
stringency_index,,,,
100.00,Suriname,2020-09-14,78.80,0.845
98.15,Argentina,2020-04-26,76.67,0.845
97.22,Sri Lanka,2021-03-29,76.98,0.804
96.30,Vietnam,2021-04-26,82.29,0.931
95.37,Madagascar,2020-04-19,67.04,0.528
94.44,Saudi Arabia,2020-08-07,82.97,0.919
93.52,Uganda,2021-03-22,83.51,0.892
92.59,Peru,2021-03-22,80.98,0.887
92.13,Kyrgyzstan,2020-09-03,76.88,0.825


In [65]:
top_lower_stringency = top_higher_stringency.tail(10)
top_lower_stringency

,location,date,life_expectancy,human_development_index
stringency_index,,,,
12.96,United Kingdom,2021-04-19,81.32,0.932
12.04,Tanzania,2021-03-23,77.29,0.767
11.11,Vietnam,2021-04-26,83.56,0.957
10.19,India,2020-03-04,69.66,0.645
9.26,Andorra,2020-11-08,83.73,0.868
8.33,United States,2021-04-12,84.63,0.955
6.48,Tanzania,2021-02-01,65.46,0.529
5.56,United States,2020-04-03,82.66,0.947
2.78,Vietnam,2021-04-26,84.63,0.929


In [61]:
top_lower_stringency = grouped_df.sort_values(by=['stringency_index'], ascending=True)
top_lower_stringency.head(10)

,location,date,life_expectancy,human_development_index
stringency_index,,,,
0.00,Vietnam,2020-03-15,83.73,0.945
2.78,Vietnam,2021-04-26,84.63,0.929
5.56,United States,2020-04-03,82.66,0.947
6.48,Tanzania,2021-02-01,65.46,0.529
8.33,United States,2021-04-12,84.63,0.955
9.26,Andorra,2020-11-08,83.73,0.868
10.19,India,2020-03-04,69.66,0.645
11.11,Vietnam,2021-04-26,83.56,0.957
12.04,Tanzania,2021-03-23,77.29,0.767
